In [1]:
!pip install -q \
    transformers==4.31.0 \
    accelerate==0.21.0 \
    bitsandbytes==0.41.0 \
    sentence-transformers==2.2.2 \
    xformers==0.0.20 \

!pip install -q \
    langchain==0.1.0 \
    langchain-community==0.0.12 \
    langchainhub==0.1.14 \
    faiss-gpu \
    faiss-cpu

!pip install -q pandas
# !pip install -q colab-xterm
!pip install -qU langchain-anthropic
!pip install -q python-dotenv

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.0 requires langchain-core<0.2,>=0.1.7, but you have langchain-core 0.2.1 which is incompatible.
langchain 0.1.0 requires langsmith<0.1.0,>=0.0.77, but you have langsmith 0.1.63 which is incompatible.
langchain-community 0.0.12 requires langchain-core<0.2,>=0.1.9, but you have langchain-core 0.2.1 which is incompatible.
langchain-community 0.0.12 requires langsmith<0.1.0,>=0.0.63, but you have langsmith 0.1.63 which is incompatible.


In [48]:
import pandas as pd
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough


from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders.dataframe import DataFrameLoader
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain_anthropic import ChatAnthropic


from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import huggingface_hub as hf_hub

try:
    from google.colab.userdata import get as getenv
except ImportError:
    from os import getenv
    import dotenv
    dotenv.load_dotenv()

In [16]:
try:
    from torch import cuda
    device = 'cuda' if cuda.is_available() else 'cpu'
except ImportError:
    device = 'cpu'

In [49]:
HF_TOKEN = getenv('HF_TOKEN')
assert HF_TOKEN, "A valid HuggingFace token is required to be set as <HF_TOKEN>."
hf_hub.login(HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/codespace/.cache/huggingface/token
Login successful


In [50]:
ANTHROPIC_API_KEY = getenv('ANTHROPIC_API_KEY')

## Constants

In [24]:
# Dataset files
PLACES_PATH = "data/places.csv"
REVIEWS_PATH = "data/reviews.csv"

# Models
MODEL_NAME =  "meta-llama/Llama-2-7b-hf" # "meta-llama/Llama-2-7b-chat-hf"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"

# Embeddings
EMBEDDINGS_CACHE_STORE="./cache/"

# Faiss
FAISS_REVIEWS_PATH = "faiss_index"
FAISS_INDEX_NAME = "index"
FAISS_DISTANCE_STRATEGY='EUCLIDEAN_DISTANCE'

## Load Dataset

Here we are using 2 csv files containing places (restuarants, bars, ...) info and reviews for each of them.

In [21]:
def get_documents(content_func=lambda row:row['review'],
                  source_func=lambda row:row['place_id'],
                  metadata_fields=[]):

  # Load both data files
  places_df = pd.read_csv(PLACES_PATH)
  reviews_df = pd.read_csv(REVIEWS_PATH)

  # merge them on 'place_id'
  merged_df = pd.merge(places_df, reviews_df, on='place_id', how='inner')

  # add page_content and source columns using their corresponing functions
  merged_df['page_content'] = merged_df.apply(content_func, axis=1)
  merged_df['source'] = merged_df.apply(source_func, axis=1)

  # update metadata_fields with 'page_content', 'source'
  metadata_fields = list(set(metadata_fields + ['page_content', 'source']))

  loader = DataFrameLoader(merged_df[metadata_fields],page_content_column='page_content')
  return loader.load()

In [25]:
def content_func(row) -> str:
  content_fields = ['place_name', 'place_types', 'place_address', 'place_average_ratings', 'review']
  return '\n'.join(f"{key}={row[key]}" for key in content_fields)

documents = get_documents(content_func)

In [26]:
documents[0]

Document(page_content='place_name=Roemerkeller Padova\nplace_types=[\'italian_restaurant\', \'steak_house\', \'pizza_restaurant\', \'meal_takeaway\', \'restaurant\', \'food\', \'point_of_interest\', \'establishment\']\nplace_address=Via Romana Aponense, 137, 35142 Padova PD, Italy\nplace_average_ratings=4.0\nreview=Huge "eating palace" which is lacking the fine accents of the italian food although the service was top. I would recommend for lunch, less for dinner', metadata={'source': 'ChIJpXSgrsDbfkcRzf_5kCMmrZI'})

## Load Embeddings model

In [27]:
def get_hf_embedding_model(embedding_model_name,
                           cache_embeddings_store,
                           device='cpu',
                           normalize_embeddings=False,
                           ):
  model_kwargs = {'device': device}
  encode_kwargs = {'normalize_embeddings': normalize_embeddings} # Set `True` for cosine similarity
  embedding_model = HuggingFaceEmbeddings(
      model_name=embedding_model_name,
      model_kwargs=model_kwargs,
      encode_kwargs=encode_kwargs
      )
  store = LocalFileStore(cache_embeddings_store)
  embedding_model = CacheBackedEmbeddings.from_bytes_store(
                    embedding_model, store)
  return embedding_model



In [28]:
embedding_model = get_hf_embedding_model(EMBEDDING_MODEL_NAME,
                                         EMBEDDINGS_CACHE_STORE,
                                         device=device,
                                         normalize_embeddings=False)

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

## Load FAISS (Vector Database)

In [30]:
def get_vector_database(documents, embedding_model):

  vector_database = FAISS.from_documents(
      documents, embedding_model,
      distance_strategy= FAISS_DISTANCE_STRATEGY
      )
  return vector_database

In [31]:
vector_db = get_vector_database(documents, embedding_model)

In [32]:
## if you want to save the db and use the files to load it again later.
vector_db.save_local(folder_path=FAISS_REVIEWS_PATH, index_name=FAISS_INDEX_NAME)
vector_db = FAISS.load_local(folder_path=FAISS_REVIEWS_PATH,
                             embeddings=embedding_model,
                             index_name=FAISS_INDEX_NAME)


In [33]:
docs = vector_db.similarity_search("which one is the best pizza restaurant in the city?", k = 5)

In [34]:
docs[0]

Document(page_content="place_name=Bar Ristorante Pizzeria Otivm Lunch Cafe'\nplace_types=['bar', 'hamburger_restaurant', 'vegan_restaurant', 'fast_food_restaurant', 'vegetarian_restaurant', 'sandwich_shop', 'american_restaurant', 'restaurant', 'food', 'point_of_interest', 'establishment']\nplace_address=Via Roma, 69, 35122 Padova PD, Italy\nplace_average_ratings=4.4\nreview=Hands down one of the BEST PIZZA I ever had! The place can get busy so don’t expect chit-chat, but totally worth it! Thank you for this culinary experience! 🙌🏻", metadata={'source': 'ChIJtSDfulHafkcRXqCEUnZKrN0'})

### Load LLM

In [52]:
def get_claude_api_llm(model_name):
  llm = ChatAnthropic(model_name=model_name, anthropic_api_key=ANTHROPIC_API_KEY,)

  return llm


In [43]:
def get_hf_llm(model_name):

  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16,
  )
  model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, )
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  pipe = pipeline(
      model=model,
      tokenizer=tokenizer,
      return_full_text=True,  # langchain expects the full text
      task='text-generation',
      # we pass model parameters here too
      temperature=0.0001,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
      max_new_tokens=512,  # mex number of tokens to generate in the output
      repetition_penalty=1.1  # without this output begins repeating
  )

  llm = HuggingFacePipeline(pipeline=pipe,)
  return llm


In [53]:
llm = get_claude_api_llm("claude-3-sonnet-20240229")

In [54]:
llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'id': 'msg_01XBLgSVfSMjQmXuagGxZNWS', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 8, 'output_tokens': 12}}, id='run-a5843c54-f15b-4e49-b9df-71c55c357345-0')

## Create LangChain pipeline

In [31]:
review_template_str = """Your job is to use Google Map
reviews to answer questions about their experience at a restaurant. Use
the following context to answer questions. Be as detailed as possible, but
don't make up any information that's not from the context. If you don't know
an answer based on the context, say you don't know.
context:
{context}
"""
## """
# If you don't know an answer based on the context, say you don't know, and
# if the context is not about restaurants, then kindly tell them that  you can
# only provide assistance and answer questions related to restaurants.
##"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
messages = [review_system_prompt, review_human_prompt]

review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)



reviews_retriever = vector_db.as_retriever(k=10,)

review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | llm
    | StrOutputParser()
)

In [32]:
# review_chain = (
#     {"context": reviews_retriever, "question": RunnablePassthrough()}
#     | review_prompt_template
# )

question = """What are the pros and cons of the best pizza restaurant in the city?"""
print(review_chain.invoke(question))

Unfortunately, there is no specific review or information provided about the "best pizza restaurant" in the city. The context has reviews for several restaurants, including a wine bar with excellent pizza, a hamburger restaurant, and a restaurant specializing in Neapolitan pizza. However, none of them are explicitly stated as being the best pizza restaurant. Without more details identifying which restaurant is considered the best for pizza, I cannot provide a detailed overview of the pros and cons based on the given information.


In [47]:
question = """What are the pros and cons of the best pizza restaurant in the city?"""
reviews_retriever.invoke(question)

# {"context": reviews_retriever, "question": RunnablePassthrough()}

# review_prompt_template

[Document(page_content="place_name=McDonald's\nplace_types=['fast_food_restaurant', 'hamburger_restaurant', 'american_restaurant', 'restaurant', 'food', 'point_of_interest', 'establishment']\nplace_address=Piazzale della Stazione, 5, 35131 Padova PD, Italy\nplace_average_ratings=3.2\nreview=I would say the services and staff are nice. The place is cozy and clean and has seats to sit and enjoy your meal.\nBut I wasn't satisfied with the taste of the food. I ordered a cheeseburger but honestly, it was not tasty as my experience in other McDonald's around Europe, Russia, and the US.\nBefore I was thinking of visiting at least once a week, but now I just visited once and not planning to visit that much soon.\n\nSince that, I am a big fan of McDonald's I always expect a better taste and products but this one was not my story of eating at McDonald's.", metadata={'source': 'ChIJndkoTl3afkcRDto-rckMt7o'}),
 Document(page_content="place_name=Enoteca Barcollo\nplace_types=['bar', 'food', 'point_

In [ ]:
question = """What are the pros and cons of the best pizza restaurant in the city?"""
review_chain.invoke(question)